In [45]:
# Librerías a utilizar

from multiprocessing import Pool
import webbrowser
import os
import pandas as pd
import numpy as np
import itertools as it
import time as time
from scipy.optimize import minimize
import warnings
warnings.filterwarnings("ignore")

#Función que maximiza el ratio de Sharpe y entrega los pesos

def daily_portfolio_return(weights, returns):
    """
    Computes the return on a portfolio from constituent returns and weights
    weights are a numpy array or Nx1 matrix and returns are a numpy array or Nx1 matrix
    """
    return (weights.T @ returns)
    
def daily_portfolio_vol(weights, covmat):
    """
    Computes the vol of a portfolio from a covariance matrix and constituent weights
    weights are a numpy array or N x 1 maxtrix and covmat is an N x N matrix
    """
    return (weights.T @ covmat @ weights)**0.5
    
def daily_neg_sharpe(weights, riskfree_rate, er, cov):
    """
    Returns the negative of the sharpe ratio
    of the given portfolio
    """
    r = daily_portfolio_return(weights, er)
    vol = daily_portfolio_vol(weights, cov)
    return -(r - riskfree_rate)/vol

def portfolio_return(weights, returns):
    """
    Computes the return on a portfolio from constituent returns and weights
    weights are a numpy array or Nx1 matrix and returns are a numpy array or Nx1 matrix
    """
    return 252*(weights.T @ returns)
    
def portfolio_vol(weights, covmat):
    """
    Computes the vol of a portfolio from a covariance matrix and constituent weights
    weights are a numpy array or N x 1 maxtrix and covmat is an N x N matrix
    """
    return (252*(weights.T @ covmat @ weights))**0.5
    
def neg_sharpe(weights, riskfree_rate, er, cov):
    """
    Returns the negative of the sharpe ratio
    of the given portfolio
    """
    r = portfolio_return(weights, er)
    vol = portfolio_vol(weights, cov)
    return -(r - riskfree_rate)/vol

def msr(riskfree_rate, er, cov):
    """
    Returns the weights of the portfolio that gives you the maximum sharpe ratio
    given the riskfree rate and expected returns and a covariance matrix
    """
    n = er.shape[0]
    init_guess = np.repeat(1/n, n)
    bounds = np.array(((0.05, 1.0),) * n) # an N-tuple of 2-tuples!
    # construct the constraints
    weights_sum_to_1 = {'type': 'eq',
                        'fun': lambda weights: np.sum(weights) - 1
    }

    weights = minimize(neg_sharpe, init_guess,
                       args=(riskfree_rate, er, cov), method='SLSQP',
                       options={'disp': False},
                       constraints=(weights_sum_to_1,),
                       bounds=bounds)
    return weights.x

# Elección de portafolio datAsset Improve


Para la optimización en la elección del portafolio ***Improve*** de **datAsset**, se escogerá el portafolio con mayor ratio de Sharpe entre todas las combinaciones posibles de portafolio de 7 acciones, que cumplan con ponderar por lo bajo **5%** cada una.

Cabe destacar que nuestra regla de decisión se basará en torno al ratio de Sharpe, el cual se calcula con la siguiente fórmula:

$$S=\frac{\mathbb{E}(r_P)-r_f}{\sigma_P}$$

Nos remitiremos solo a las acciones que nos permitan obtener un Ratio de Sharpe positivo, por lo que eliminaremos todas las acciones cuyos retornos logarítmicos diarios promedio los dos primeros años sean menores al retorno diario del activo libre de riesgo.

Escogeremos nuestros activos del **S&P100**, y en NYSE se transan aprox. 252 días al año, nuestro retorno diario para el activo libre de riesgo sería **2.7%**. En el caso de querer hacer cálculos diarios se divide por la cantidad de días transables.

In [46]:
# Calculo activo libre de riesgo

risk_free_asset = [0.027 , 0]
risk_free = [0.027/252 , 0]

Ahora que ya tenemos definido nuestro problema, debemos obtener los datos para esto el siguiente script se encarga de descargar todos los datos de Yahoo Finance para las fechas entre el **01 de Enero del 2017** y **31 de Diciembre del 2019**.

In [47]:
# Se descargan todos los históricos del SP&100

assets = ["AAPL", "ABBV", "ABT", "ACN", "ADBE", "AIG", "ALL", "AMGN", "AMT", 
              "AMZN", "AXP", "BA", "BAC", "BIIB", "BK", "BKNG", "BLK", "BMY", 
              "C", "CAT", "CHTR", "CL", "CMCSA", "COF", "COP", "COST", "CRM", 
              "CSCO", "CVS", "CVX", "DD", "DHR", "DIS", "DOW", "DUK", "EMR", 
              "EXC", "F", "FB", "FDX", "GD", "GE", "GILD", "GM", "GOOG", "GOOGL", 
              "GS", "HD", "HON", "IBM", "INTC", "JNJ", "JPM", "KHC", "KMI", "KO",
              "LLY", "LMT", "LOW", "MA", "MCD", "MDLZ", "MDT", "MET", "MMM", "MO", 
              "MRK", "MS", "MSFT", "NEE", "NFLX", "NKE", "NVDA", "ORCL", "OXY", 
              "PEP", "PFE", "PG", "PM", "PYPL", "QCOM", "RTX", "SBUX", "SLB", 
              "SO", "SPG", "T", "TGT", "TMO", "TXN", "UNH", "UNP", "UPS", "USB", 
              "V", "VZ", "WBA", "WFC", "WMT", "XOM"]

'''
for i in assets:
    link = "https://query1.finance.yahoo.com/v7/finance/download/{}?period1=1483228800&period2=1577750400&interval=1d&events=history".format(i)
    webbrowser.open(link)
'''

'\nfor i in assets:\n    link = "https://query1.finance.yahoo.com/v7/finance/download/{}?period1=1483228800&period2=1577750400&interval=1d&events=history".format(i)\n    webbrowser.open(link)\n'

### Análisis de los activos

Ahora analizamos que todos los activos lleven la misma cantidad de días transandose, para no tener problemas más adelante con el cálculo de los retornos.

In [48]:
df_shape = pd.DataFrame(index=assets, columns=["shape"])

for filename in os.listdir('data/assets'):
    file = open('data/assets/'+filename)
    df_shape.loc[os.path.basename(file.name)[:-4]]["shape"] = pd.read_csv('data/assets/'+filename).shape[0]
    
df_shape[df_shape["shape"]<753]

,shape
DOW,198


Se elimina *Dow Inc (DOW)* por tener solo 198 días transando en el mercado. Además se leen todos los precios de los activos y se crea un dataframe con los cierres ajustados.

In [58]:
# Se elimina DOW y se genera un dataframe con cierres ajsutados

assets = ["AAPL", "ABBV", "ABT", "ACN", "ADBE", "AIG", "ALL", "AMGN", "AMT", 
              "AMZN", "AXP", "BA", "BAC", "BIIB", "BK", "BKNG", "BLK", "BMY", 
              "C", "CAT", "CHTR", "CL", "CMCSA", "COF", "COP", "COST", "CRM", 
              "CSCO", "CVS", "CVX", "DD", "DHR", "DIS", "DUK", "EMR", 
              "EXC", "F", "FB", "FDX", "GD", "GE", "GILD", "GM", "GOOG", "GOOGL", 
              "GS", "HD", "HON", "IBM", "INTC", "JNJ", "JPM", "KHC", "KMI", "KO",
              "LLY", "LMT", "LOW", "MA", "MCD", "MDLZ", "MDT", "MET", "MMM", "MO", 
              "MRK", "MS", "MSFT", "NEE", "NFLX", "NKE", "NVDA", "ORCL", "OXY", 
              "PEP", "PFE", "PG", "PM", "PYPL", "QCOM", "RTX", "SBUX", "SLB", 
              "SO", "SPG", "T", "TGT", "TMO", "TXN", "UNH", "UNP", "UPS", "USB", 
              "V", "VZ", "WBA", "WFC", "WMT", "XOM"]

Maestro = pd.read_csv('data/assets/AAPL.csv')[["Date"]]

for asset in assets:
    A = pd.read_csv('data/assets/'+asset+".csv")
    Maestro[asset] = A['Adj Close']

Maestro

,Date,AAPL,ABBV,ABT,ACN,ADBE,AIG,ALL,AMGN,AMT,...,UNH,UNP,UPS,USB,V,VZ,WBA,WFC,WMT,XOM
0,2017-01-03,110.392334,52.551662,36.556423,109.816139,103.480003,59.218830,69.386414,136.338608,99.202438,...,152.995758,94.916626,102.347374,46.466454,77.675957,46.683941,75.488945,49.328472,63.187412,77.590286
1,2017-01-04,110.268791,53.292667,36.846622,110.080170,104.139999,59.991787,69.573509,138.274292,99.380005,...,153.431671,95.490654,102.374054,46.809586,78.311043,46.632622,75.507172,49.372513,63.555534,76.736603
2,2017-01-05,110.829552,53.696846,37.164909,108.430000,105.910004,59.655315,69.255440,138.373795,99.034233,...,153.687500,94.555565,102.427414,46.024002,79.229477,46.735260,75.552658,48.606167,63.693573,75.592690
3,2017-01-06,112.065109,53.713688,38.175949,109.665268,108.300003,60.755657,69.367714,141.810974,98.380020,...,153.905472,95.536949,102.631989,46.321983,80.323792,46.041534,75.616364,48.482838,62.819294,75.550018
4,2017-01-09,113.091560,54.067345,38.138512,108.439430,108.570000,60.364632,68.376076,143.674286,98.146400,...,153.469559,94.824051,102.071663,46.114304,79.874336,45.540134,75.115883,47.778152,63.233440,74.303650
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
748,2019-12-23,282.562683,87.738403,86.615829,209.074692,328.950012,49.825058,109.725632,239.694580,224.082764,...,292.557281,177.374725,116.550629,57.785435,186.497177,60.121017,57.411610,52.181675,117.986633,67.977135
749,2019-12-24,282.831299,87.349533,86.546417,209.848190,329.640015,49.795727,110.012611,239.004181,224.191360,...,292.011993,177.839371,115.952888,57.872723,186.985657,60.003517,57.195957,52.191368,118.462433,67.716011
750,2019-12-26,288.442780,87.330093,86.546417,210.284531,331.200012,49.482922,110.438133,238.580078,225.976379,...,293.112457,178.748871,116.648621,58.338264,188.570694,60.013309,57.735085,52.511387,118.472336,67.822395
751,2019-12-27,288.333313,86.717621,86.665413,210.453110,330.790009,49.922810,110.705330,238.215164,227.176346,...,293.429718,179.342056,116.550629,57.930916,188.799973,60.248306,57.852707,52.288342,118.541725,67.590294


### Cálculo de retornos

Se calculan los retornos logaritmicos con el siguiente scripr, para esto se utiliza la fórmula

$$r_t=\ln\left(\frac{p_t}{p_{t-1}}\right)$$

Además creamos variables para el año el día y la fecha.

In [50]:
# Se calculan los retornos logarítimicos y retorno libre de riesgo

ret = Maestro.iloc[1:][assets]/Maestro.iloc[:-1][assets].values
log_ret = ret.apply(np.log)
log_ret.Date = Maestro.Date
log_ret.Date = pd.to_datetime(log_ret.Date)
log_ret[['year','month','day']] = log_ret.Date.apply(lambda x: pd.Series(x.strftime("%Y,%m,%d").split(",")))
log_ret

,AAPL,ABBV,ABT,ACN,ADBE,AIG,ALL,AMGN,AMT,AMZN,...,USB,V,VZ,WBA,WFC,WMT,XOM,year,month,day
1,-0.001120,0.014002,0.007907,0.002401,0.006358,0.012968,0.002693,0.014098,0.001788,0.004646,...,0.007357,0.008143,-0.001100,0.000241,0.000892,0.005809,-0.011063,2017,01,04
2,0.005073,0.007556,0.008601,-0.015104,0.016854,-0.005624,-0.004582,0.000719,-0.003485,0.030270,...,-0.016925,0.011660,0.002199,0.000602,-0.015643,0.002170,-0.015019,2017,01,05
3,0.011087,0.000314,0.026841,0.011328,0.022315,0.018277,0.001620,0.024536,-0.006628,0.019716,...,0.006454,0.013717,-0.014955,0.000843,-0.002541,-0.013821,-0.000565,2017,01,06
4,0.009118,0.006563,-0.000981,-0.011241,0.002490,-0.006457,-0.014399,0.013054,-0.002377,0.001168,...,-0.004493,-0.005611,-0.010950,-0.006641,-0.014641,0.006571,-0.016635,2017,01,09
5,0.001008,-0.002183,0.013410,0.000522,-0.002859,0.012277,0.008989,-0.000504,-0.014579,-0.001281,...,0.002933,-0.005397,0.001517,0.001453,0.006981,-0.007010,-0.012835,2017,01,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
748,0.016187,0.010694,0.007931,-0.001280,0.004082,-0.011121,-0.006293,-0.000123,-0.003387,0.003632,...,-0.003184,-0.004906,-0.010853,0.003078,0.008960,-0.010530,0.004992,2019,12,23
749,0.000950,-0.004442,-0.000802,0.003693,0.002095,-0.000589,0.002612,-0.002884,0.000485,-0.002116,...,0.001509,0.002616,-0.001956,-0.003763,0.000186,0.004025,-0.003849,2019,12,24
750,0.019646,-0.000223,0.000000,0.002077,0.004721,-0.006302,0.003860,-0.001776,0.007931,0.043506,...,0.008012,0.008441,0.000163,0.009382,0.006113,0.000084,0.001570,2019,12,26
751,-0.000380,-0.007038,0.001374,0.000801,-0.001239,0.008850,0.002417,-0.001531,0.005296,0.000551,...,-0.007007,0.001215,0.003908,0.002035,-0.004257,0.000586,-0.003428,2019,12,27


Lo que haremos ahora será particionar la tabla de retornos logarítmicos en 2, una tabla para los dos primeros años que llamaremos **log_ret_f2y** y una tabla para el último año que llamaremos **log_ret_ly**. 

In [51]:
# Se particionen los retornos logaritmicos de los dos primeros años y los del último año

log_ret_f2y = log_ret[(log_ret['year']=='2017') | (log_ret['year']=='2018')]
log_ret_ly = log_ret[(log_ret['year']=='2019')]

### Regla de exclusión

Ahora se eliminaran los activos con un retorno menor que el retorno del activo libre de riesgo, para disminuir el conjunto objetivo en nuestra futura optimización. Para esto calculamos los retornos promedio y desviaciones para ambas particiones, e imponemos el filtro para los activos cuyo retorno promedio para ambas particiones sea mayor al del activo libre de riesgo.

In [52]:
# Se calculan los retornos promedios y desviaciones para ambas particiones

ret_mean_f2y = log_ret_f2y.mean()
ret_std_f2y = log_ret_f2y.std()

ret_mean_ly = log_ret_ly.mean()
ret_std_ly = log_ret_ly.std()

log_ret_ly = log_ret_ly[(ret_mean_f2y[ret_mean_f2y>risk_free[0]].index) & (ret_mean_ly[ret_mean_ly>risk_free[0]].index)]

temp_assets = log_ret_ly.mean().sort_values(ascending=False)[3:]
np.array(temp_assets.index)

array(['AAPL', 'NVDA', 'MA', 'MSFT', 'FB', 'LMT', 'ACN', 'DHR', 'AMT',
       'JPM', 'COST', 'BAC', 'TMO', 'ADBE', 'V', 'NEE', 'PG', 'TXN',
       'SBUX', 'ALL', 'NKE', 'HON', 'UNP', 'DIS', 'LOW', 'AXP', 'EMR',
       'BLK', 'WMT', 'INTC', 'HD', 'GOOG', 'PYPL', 'GOOGL', 'PEP', 'AMGN',
       'MDT', 'AMZN', 'MRK', 'ABT', 'NFLX', 'KO', 'UNH', 'CAT', 'BKNG',
       'ORCL', 'CRM', 'LLY', 'JNJ', 'MCD', 'VZ', 'CSCO', 'DUK', 'COP',
       'EXC', 'BA', 'ABBV'], dtype=object)

Logramos reducir a 57 los activos cuyo retorno promedio diario en ambas particiones fuera mayor al activo libre de riesgo. Ahora queremos ver cuantos portafolios de entre 6 a 10 activos podemos construir con estos, la respuesta sería:

$$N=\sum_{k=6}^{10}\frac{57!}{(57-k)!\cdot k!}=54132567918$$

Tenemos cerca de **54 billones** de portafolios, y a cada uno de estos portafolios debemos optimizar su ponderación para obtener así el portafolio con mayor ratio de Sharpe. Es decir, debemos realizar 54 billones de optimizaciones con algún metodo numérico, por lo que el computo tardaría si lo hacemos en un computador convencional. No utilizaremos la fuerza bruta, es más, utilizaremos la convexidad de nuestro problema para plantear un problema alternativo que también nos entrega el mayor ratio de Sharpe.

### Optimización recursiva

Consideremos un portafolio $P_{57}$, compuesto por las $57$ acciones que nos cumplen el problema, calcularemos un portafolio compuesto por $56$ de entre esas $57$ que maximize el ratio de Sharpe, luego de esas $56$ acciones escojeremos el portafolio con $55$ acciones que maximize el ratio de Sharpe, y asi recursivamente hasta llegar al portafolio de entre 6 a 10 que nos maximize este ratio. Dada la convexidad del problema de optimización, podemos afirmar que así encontraremos el portafolio óptimo.

In [56]:
# Algoritmo para la optimización recursiva

filtered_assets = np.array(temp_assets.index)

n = len(temp_assets)-1

max_sharpe = ["Assets","Weights","Sharpe_ratio"]
max_sharpe = ["Assets","Weights",0]

while n>=6:
  
    options = it.combinations(filtered_assets,n)
    
    for subset in options:

        subset = list(subset)

        cov = log_ret_f2y[subset].cov()
        ret = ret_mean_f2y[subset].values
        ass = ret_mean_f2y[subset].index.values

        if ret.shape[0]==cov.shape[0]:
            weights = msr(risk_free_asset[0],ret,cov)

            if max_sharpe[2]<-neg_sharpe(weights, risk_free_asset[0], ret, cov):
                max_sharpe[0] = subset
                max_sharpe[1] = weights
                max_sharpe[2] = -neg_sharpe(weights, risk_free_asset[0], ret, cov)
            else: pass
        else: pass
    print(max_sharpe[0])
    filtered_assets = max_sharpe[0]
    n-=1
    

['AAPL', 'MA', 'MSFT', 'FB', 'LMT', 'ACN', 'DHR', 'AMT', 'JPM', 'COST', 'BAC', 'TMO', 'ADBE', 'V', 'NEE', 'PG', 'TXN', 'SBUX', 'ALL', 'NKE', 'HON', 'UNP', 'DIS', 'LOW', 'AXP', 'EMR', 'BLK', 'WMT', 'INTC', 'HD', 'GOOG', 'PYPL', 'GOOGL', 'PEP', 'AMGN', 'MDT', 'AMZN', 'MRK', 'ABT', 'NFLX', 'KO', 'UNH', 'CAT', 'BKNG', 'ORCL', 'CRM', 'LLY', 'JNJ', 'MCD', 'VZ', 'CSCO', 'DUK', 'COP', 'EXC', 'BA', 'ABBV']
['AAPL', 'MA', 'MSFT', 'FB', 'LMT', 'ACN', 'DHR', 'AMT', 'JPM', 'COST', 'BAC', 'TMO', 'ADBE', 'V', 'NEE', 'PG', 'TXN', 'SBUX', 'ALL', 'NKE', 'HON', 'UNP', 'DIS', 'LOW', 'AXP', 'EMR', 'WMT', 'INTC', 'HD', 'GOOG', 'PYPL', 'GOOGL', 'PEP', 'AMGN', 'MDT', 'AMZN', 'MRK', 'ABT', 'NFLX', 'KO', 'UNH', 'CAT', 'BKNG', 'ORCL', 'CRM', 'LLY', 'JNJ', 'MCD', 'VZ', 'CSCO', 'DUK', 'COP', 'EXC', 'BA', 'ABBV']
['AAPL', 'MA', 'MSFT', 'LMT', 'ACN', 'DHR', 'AMT', 'JPM', 'COST', 'BAC', 'TMO', 'ADBE', 'V', 'NEE', 'PG', 'TXN', 'SBUX', 'ALL', 'NKE', 'HON', 'UNP', 'DIS', 'LOW', 'AXP', 'EMR', 'WMT', 'INTC', 'HD', 'GOOG',

['MA', 'MSFT', 'AMT', 'COST', 'TMO', 'ADBE', 'V', 'NEE', 'PG', 'SBUX', 'NKE', 'UNP', 'WMT', 'PYPL', 'MDT', 'AMZN', 'MRK', 'ABT', 'NFLX', 'KO', 'UNH', 'CRM', 'LLY', 'MCD', 'VZ', 'DUK', 'EXC', 'BA', 'ABBV']
['MA', 'MSFT', 'AMT', 'COST', 'TMO', 'ADBE', 'V', 'NEE', 'PG', 'NKE', 'UNP', 'WMT', 'PYPL', 'MDT', 'AMZN', 'MRK', 'ABT', 'NFLX', 'KO', 'UNH', 'CRM', 'LLY', 'MCD', 'VZ', 'DUK', 'EXC', 'BA', 'ABBV']
['MA', 'AMT', 'COST', 'TMO', 'ADBE', 'V', 'NEE', 'PG', 'NKE', 'UNP', 'WMT', 'PYPL', 'MDT', 'AMZN', 'MRK', 'ABT', 'NFLX', 'KO', 'UNH', 'CRM', 'LLY', 'MCD', 'VZ', 'DUK', 'EXC', 'BA', 'ABBV']
['MA', 'AMT', 'COST', 'TMO', 'ADBE', 'V', 'NEE', 'PG', 'NKE', 'UNP', 'WMT', 'PYPL', 'MDT', 'AMZN', 'MRK', 'ABT', 'NFLX', 'KO', 'UNH', 'CRM', 'LLY', 'MCD', 'DUK', 'EXC', 'BA', 'ABBV']
['MA', 'AMT', 'COST', 'TMO', 'ADBE', 'V', 'NEE', 'PG', 'NKE', 'UNP', 'WMT', 'PYPL', 'AMZN', 'MRK', 'ABT', 'NFLX', 'KO', 'UNH', 'CRM', 'LLY', 'MCD', 'DUK', 'EXC', 'BA', 'ABBV']
['MA', 'AMT', 'COST', 'TMO', 'ADBE', 'V', 'NEE', '

In [57]:
print("La cartera que máximiza el ratio de Sharpe es: ")
print(max_sharpe[0])
print("")
print("Los pesos respectivos son: ")
print(max_sharpe[1])
print("")
print("El ratio de Sharpe es: ")
print(max_sharpe[2])

La cartera que máximiza el ratio de Sharpe es: 
['AMT', 'ADBE', 'NEE', 'ABT', 'LLY', 'MCD', 'BA']

Los pesos respectivos son: 
[0.05119548 0.05       0.34249054 0.20630113 0.08208151 0.12308989
 0.14484145]

El ratio de Sharpe es: 
2.1788700029587065


## Código Fuerza Bruta

El siguiente código corría la optimización con fuerza bruta, nos dimos cuenta que la optimización recursiva reducía el orden del algoritmo en muchos grados, por lo que no se recomienda correr este código

In [9]:
# Ejemplo del optimizador para escoger el óptimo de 3 activos entre 5

# filtered_assets = temp_assets
filtered_assets = ['AAPL', 'ABBV', 'ABT','ACN','UNH']
n = 3

tiempo = time.time()
max_sharpe = ["Assets","Weights","Sharpe_ratio"]
max_sharpe = ["Assets","Weights",0]

options = it.combinations(filtered_assets,n)

start = time.time()

for subset in options:

    subset = list(subset)
    
    cov = log_ret_f2y[subset].cov()
    ret = ret_mean_f2y[subset].values
    ass = ret_mean_f2y[subset].index.values
    
    if ret.shape[0]==cov.shape[0]:
        weights = msr(risk_free_asset[0],ret,cov)

        if max_sharpe[2]<-neg_sharpe(weights, risk_free_asset[0], ret, cov):
            max_sharpe[0] = subset
            max_sharpe[1] = weights
            max_sharpe[2] = -neg_sharpe(weights, risk_free_asset[0], ret, cov)
        else: pass
    else: pass

end = time.time()
print(end - start)
print("La cartera que máximiza el ratio de Sharpe es: ")
print(max_sharpe[0])
print("")
print("Los pesos respectivos son: ")
print(max_sharpe[1])
print("")
print("El ratio de Sharpe es: ")
print(max_sharpe[2])

0.2693789005279541
La cartera que máximiza el ratio de Sharpe es: 
['ABBV', 'ABT', 'UNH']

Los pesos respectivos son: 
[0.05       0.80766278 0.14233722]

El ratio de Sharpe es: 
1.6343513704636377


In [37]:
# Optimizador, se recomienda no ejecutar este código; es demasiado procesamiento. Se ejecutara en GCP

'''
filtered_assets = np.array(['MA', 'AMT', 'TMO', 'ADBE', 'NEE', 'UNP'])

max_sharpe = ["Assets","Weights","Sharpe_ratio"]
max_sharpe = ["Assets","Weights",0]

k=1


for subset in it.combinations(filtered_assets,6):
    subset = list(subset)

    cov = log_ret_f2y[subset].cov()
    ret = ret_mean_f2y[subset].values
    ass = ret_mean_f2y[subset].index.values

    if ret.shape[0]==cov.shape[0]:
        weights = msr(risk_free_asset[0],ret,cov)

        if max_sharpe[2]<-neg_sharpe(weights, risk_free_asset[0], ret, cov):
            max_sharpe[0] = subset
            max_sharpe[1] = weights
            max_sharpe[2] = -neg_sharpe(weights, risk_free_asset[0], ret, cov)
        else: pass
        if k%1000==0: 
            print(k)
            print("La cartera que máximiza el ratio de Sharpe es: ")
            print(max_sharpe[0])
            print("")
            print("Los pesos respectivos son: ")
            print(max_sharpe[1])
            print("")
            print("El ratio de Sharpe es: ")
            print(max_sharpe[2])
        else: pass
        k+=1
        
print("La cartera que máximiza el ratio de Sharpe es: ")
print(max_sharpe[0])
print("")
print("Los pesos respectivos son: ")
print(max_sharpe[1])
print("")
print("El ratio de Sharpe es: ")
print(max_sharpe[2])
'''

La cartera que máximiza el ratio de Sharpe es: 
['MA', 'AMT', 'TMO', 'ADBE', 'NEE', 'UNP']

Los pesos respectivos son: 
[0.11999747 0.12047171 0.06790822 0.14053309 0.5010895  0.05      ]

El ratio de Sharpe es: 
1.8216806850374145
